In [18]:
import pandas as pd
import openai
import os
from annotation_utils import get_assistant_texts, get_message_ids, get_prompter_texts, split_convs
from AptTagger import AptTagger
from TruthLabeler import TruthLabeler

openai.api_key_path = "/home/alex/.personal/openAIkey"  # read protected to my 174 account

In [19]:
texts = []
for path in os.listdir("data/oasst/transcripts/original"):
    with open(os.path.join("data/oasst/transcripts/original", path)) as f:
        texts.append(f.read())

text = "\n\n\n".join(texts)
raw_df = pd.DataFrame()
raw_df["message_id"] = get_message_ids(text)
raw_df["user_text"] = get_prompter_texts(text)
raw_df["assitant_text"] = get_assistant_texts(text)
raw_df.index = raw_df["message_id"]
print(len(raw_df))

7550


In [20]:
apt_tagger = AptTagger(
    model_name="data/apt-tagger-deberta-xlarge-mnli-s63day9p",
    device="cuda"
)

In [21]:
raw_df = raw_df.iloc[:10]

In [22]:
raw_df["tag_mask"] = apt_tagger.tag(raw_df["assitant_text"])
raw_df["annotated_assistant_text"] = raw_df.apply(lambda x: apt_tagger.annotate(x["assitant_text"], x["tag_mask"]), axis=1)
raw_df["annotated_transcript"] = raw_df.apply(lambda x: f"USER: {x['user_text']}\n\nASSISTANT: {x['annotated_assistant_text']}", axis=1)
raw_df.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (579 > 512). Running this sequence through the model will result in indexing errors


,message_id,user_text,assitant_text,tag_mask,annotated_assistant_text,annotated_transcript
message_id,,,,,,
MESSAGE 6d275740-274d-46ce-85db-cf72d5737bac,MESSAGE 6d275740-274d-46ce-85db-cf72d5737bac,Could you give me ISO-8061 time duration perio...,ISO 8061 is a international standard that defi...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ISO 8061 is a international standard that defi...,USER: Could you give me ISO-8061 time duration...
d3cd6431-3e58-4c04-a617-6d6c48908d57,d3cd6431-3e58-4c04-a617-6d6c48908d57,Can you write a rap song on why open assistant...,"I'm sorry, but I am unable to assist with that...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","I'm sorry, but I am unable to assist with that...",USER: Can you write a rap song on why open ass...
cc980f3a-74b6-4335-93ee-bf889299eade,cc980f3a-74b6-4335-93ee-bf889299eade,Where was the bomb dropped in Hiroshima?,The bomb was dropped on the city of Hiroshima....,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The bomb was dropped on the city of Hiroshima[...,USER: Where was the bomb dropped in Hiroshima?...
60b34a95-70db-4263-a5de-8ce86d354792,60b34a95-70db-4263-a5de-8ce86d354792,What materials are best for making a terrarium?,The best materials for making a terrarium are ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The best materials for making a terrarium are ...,USER: What materials are best for making a ter...
17058165-5004-4c68-aada-51bc656ff9ee,17058165-5004-4c68-aada-51bc656ff9ee,How many trees were there in the Garden of Eden?,The Bible doesn’t say how many trees there wer...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The Bible doesn’t say how many trees there wer...,USER: How many trees were there in the Garden ...


In [23]:
score_to_p_apt = {
    "N/A": 0,
    "Truth-apt - Certainly True": 1,
    "Truth-apt - True": 1,
    "Truth-apt - Uncertain": 0.3,
    "Truth-apt - False": 1,
    "Truth-apt - Certainly False": 1,
}
score_to_p_true = {
    "N/A": 0.5,
    "Truth-apt - Certainly True": 1,
    "Truth-apt - True": 0.8,
    "Truth-apt - Uncertain": 0.5,
    "Truth-apt - False": 0.2,
    "Truth-apt - Certainly False": 0,
}
truth_labeler = TruthLabeler(
                model_name="gpt-3.5-turbo",
                temperature=1,
                n_samples=5,
                uncertainty_bias=0,
                na_bias=0,
                score_to_p_apt=score_to_p_apt,
                score_to_p_true=score_to_p_true,
)

In [24]:
out_df = truth_labeler.label(raw_df["message_id"], raw_df["annotated_transcript"], n_threads=10)

SKIPPING: scores must be one of ['N/A', 'Truth-apt - Certainly True', 'Truth-apt - True', 'Truth-apt - Uncertain', 'Truth-apt - False', 'Truth-apt - Certainly False'], but found ['[[1]] Truth-apt - Certainly True', '[[2]] Truth-apt - True']
SKIPPING: scores must be one of ['N/A', 'Truth-apt - Certainly True', 'Truth-apt - True', 'Truth-apt - Uncertain', 'Truth-apt - False', 'Truth-apt - Certainly False'], but found ['[[1]] Truth-apt - Uncertain', '[[2]] Truth-apt - True']
SKIPPING: scores must be one of ['N/A', 'Truth-apt - Certainly True', 'Truth-apt - True', 'Truth-apt - Uncertain', 'Truth-apt - False', 'Truth-apt - Certainly False'], but found ['[[1]] Truth-apt - True', '[[2]] Truth-apt - True', '[[3]] Truth-apt - True']
SKIPPING: scores must be one of ['N/A', 'Truth-apt - Certainly True', 'Truth-apt - True', 'Truth-apt - Uncertain', 'Truth-apt - False', 'Truth-apt - Certainly False'], but found ['[[1]] Truth-apt - False', '[[2]] Truth-apt - True', '[[3]] Truth-apt - True', '[[4]] T

In [27]:
labeled_df = out_df.copy()
labeled_df.index = out_df["message_id"]
labeled_df = labeled_df.drop(columns=["message_id", "annotated_transcript"])
df = labeled_df.join(raw_df, how="left")
df.head()

,idx,input,responses,prompt_tokens,completion_tokens,dollars,pred_scores,p_apts,p_trues,ann_count,message_id,user_text,assitant_text,tag_mask,annotated_assistant_text,annotated_transcript
message_id,,,,,,,,,,,,,,,,
907c560c-43df-4c24-ab28-0e922b53bc01,0,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,[[[1]] Score: Truth-apt - True\n[[2]] Score: T...,259,102,0.000593,"[(Truth-apt - True, Truth-apt - True, Truth-ap...","[1.0, 1.0]","[0.8, 0.8]",2,907c560c-43df-4c24-ab28-0e922b53bc01,Where is the capital of Colombia?,The capital of Colombia is Bogota. It is locat...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The capital of Colombia is Bogota[[1]]. It is ...,USER: Where is the capital of Colombia?\n\nASS...
ded386df-c893-4f51-bb89-8c4c060fde38,0,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,[[[1]] Score: Truth-apt - Uncertain\n[[2]] Sco...,298,106,0.000659,"[(Truth-apt - Uncertain, Truth-apt - Uncertain...","[0.3, 1.0]","[0.5, 0.8]",2,ded386df-c893-4f51-bb89-8c4c060fde38,What visa options do I have if I want to live ...,"Depending on the country you are considering, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Depending on the country you are considering, ...",USER: What visa options do I have if I want to...
d3cd6431-3e58-4c04-a617-6d6c48908d57,0,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,[[[1]] Score: Truth-apt - True\n[[2]] Score: N...,303,97,0.000648,"[(Truth-apt - True, N/A, Truth-apt - True, Tru...","[0.8, 0.6]","[0.5800000000000001, 0.68]",2,d3cd6431-3e58-4c04-a617-6d6c48908d57,Can you write a rap song on why open assistant...,"I'm sorry, but I am unable to assist with that...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","I'm sorry, but I am unable to assist with that...",USER: Can you write a rap song on why open ass...
cc980f3a-74b6-4335-93ee-bf889299eade,0,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,[[[1]] Score: Truth-apt - True\n[[2]] Score: T...,281,154,0.000730,"[(Truth-apt - True, Truth-apt - True, Truth-ap...","[1.0, 1.0, 1.0]","[0.8, 0.8, 0.8]",3,cc980f3a-74b6-4335-93ee-bf889299eade,Where was the bomb dropped in Hiroshima?,The bomb was dropped on the city of Hiroshima....,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",The bomb was dropped on the city of Hiroshima[...,USER: Where was the bomb dropped in Hiroshima?...
b809ceff-be37-4fe0-8993-5477f30c2fc3,0,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,[[[1]] Score: Truth-apt - Uncertain\n[[2]] Sco...,279,162,0.000742,"[(Truth-apt - Uncertain, Truth-apt - True, Tru...","[0.86, 0.86, 1.0]","[0.74, 0.74, 0.8]",3,b809ceff-be37-4fe0-8993-5477f30c2fc3,I have a question about the Roman poets. Where...,"Virgil was born in Rome, but he lived most of ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Virgil was born in Rome[[1]], but he lived mos...",USER: I have a question about the Roman poets....
